<a href="https://colab.research.google.com/github/JustCal1MeLee/cnn_skin-classfication_makemodel_practice/blob/main/skin_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import torch
import torchvision
import torch.nn as nn
# import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset

import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
# import os 

In [ ]:
learning_rate = 0.01
traing_epoch = 3
batch_size = 30

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device) 

In [ ]:
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
database_path = '/content/drive/MyDrive/DATABASE'
# the database folder have a classification label folder

In [ ]:
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

trans = transforms.Compose([
                            transforms.Resize([300,300]),
                            transforms.ToTensor(),
                            transforms.RandomHorizontalFlip(p=0.5),
                            transforms.RandomVerticalFlip(p=0.5),  
                            transforms.Normalize(torch.Tensor(mean),torch.Tensor(std))
])
# 데이터늘리기위해 randomhorizontalflip , verticalflip 활용 
train_data = torchvision.datasets.ImageFolder(root = database_path, transform = trans)
test_data = torchvision.datasets.ImageFolder(root = database_path, transform= trans)
# imagefolder 함수 이용시 편하게 label활용가능

In [ ]:
len(train_data)

In [ ]:
print(train_data.classes)

In [ ]:
# 알맞게 됐나 확인
def show_transformed_images(dataset):
    loader = DataLoader(dataset, batch_size= 6, shuffle = True)
    batch = next(iter(loader))
    images, labels = batch

    grid = torchvision.utils.make_grid(images, nrow= 3)
    plt.figure(figsize=(11,11))
    plt.imshow(np.transpose(grid, (1,2,0)))
    print('labels :', labels)

In [ ]:
show_transformed_images(train_data)

In [ ]:
train_loader = DataLoader(train_data , batch_size=batch_size, shuffle= True, 
                             drop_last=True)
test_loader = DataLoader(test_data , batch_size=batch_size, shuffle=True, 
                             drop_last=True)

In [ ]:
def train_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs):
    for epoch in range(n_epochs):
        print("Epoch number %d" % (epoch+1))
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        
        for data in train_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)

            optimizer.zero_grad()

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            running_correct += (labels == predicted).sum().item()

        epoch_loss = running_loss/len(train_loader)
        epoch_acc = 100.00 * running_correct / total

        print( " -training dataset, Got %d out of %d images correctly (%.3f%%), Epoch loss : %.3f" 
              %(running_correct, total, epoch_acc, epoch_loss))
        evaluate_model_on_test_set(model, test_loader)

    print("finished")
    return model

In [ ]:
def evaluate_model_on_test_set(model, test_loader):
    model.eval()
    predicted_correctly_on_epoch = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            predicted_correctly_on_epoch += (predicted == labels).sum().item()
    epoch_acc = 100.0 * predicted_correctly_on_epoch / total
    print( " -Testing dataset, Got %d out of %d images correctly (%.3f%%)" 
              %(predicted_correctly_on_epoch, total, epoch_acc))

In [ ]:
resnet18_model = torchvision.models.resnet18(pretrained=True)
num_features = resnet18_model.fc.in_features
number_of_classes = 4
resnet18_model.fc = nn.Linear(num_features, number_of_classes)
resnet18_model = resnet18_model.to(device)

loss_fn = nn.CrossEntropyLoss()

optimizer = optim.SGD(resnet18_model.parameters(), lr = learning_rate, momentum= 0.9)

In [ ]:
train_nn(resnet18_model, train_loader, test_loader, loss_fn, optimizer, 20)

In [ ]:
# save model
model_path = '/content/drive/MyDrive/model/model.pt'
torch.save(resnet18_model.state_dict(), model_path)

https://stackoverflow.com/questions/53612835/size-mismatch-for-fc-bias-and-fc-weight-in-pytorch

In [ ]:
# if loaded
model_path ='/content/drive/MyDrive/model/model.pt'
model = torchvision.models.resnet18(pretrained = True)
num_features = model.fc.in_features
number_of_classes = 4
model.fc = nn.Linear(num_features, number_of_classes)
model = model.to(device)

model.load_state_dict(torch.load(model_path))
model.eval()